In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
import toppra as ta
import toppra.algorithm as algo
import toppra.constraint as constraint

from pydrake.all import (
    DiagramBuilder, RigidTransform, RollPitchYaw,
    Box, Cylinder, CoulombFriction,
    MultibodyPlant, SceneGraph, Meshcat, MeshcatVisualizer,
    PiecewisePose, RotationMatrix
)

from pydrake.planning import RrtConnect  # Note the capitalization: RrtConnect, not RRTConnect

In [ ]:

# === Build the diagram with a cube and cylinder ===
builder = DiagramBuilder()
plant, scene_graph = MultibodyPlant(time_step=0.0), SceneGraph()
plant.RegisterAsSourceForSceneGraph(scene_graph)
builder.AddSystem(plant)
builder.AddSystem(scene_graph)

# Add cube (free-floating)
cube_shape = Box(0.1, 0.1, 0.1)
cube_body = plant.AddRigidBody("cube", plant.GetDefaultRigidBodyFrame())
plant.RegisterCollisionGeometry(cube_body, RigidTransform(), cube_shape, "cube_collision", CoulombFriction(0.9, 0.8))
plant.RegisterVisualGeometry(cube_body, RigidTransform(), cube_shape, "cube_visual", [1, 0, 0, 1])

# Add cylinder obstacle
cylinder = Cylinder(0.2, 1.0)
X_O_cylinder = RigidTransform([0.5, 0, 0.5])
plant.RegisterCollisionGeometry(plant.world_body(), X_O_cylinder, cylinder, "cylinder_collision", CoulombFriction(0.9, 0.8))
plant.RegisterVisualGeometry(plant.world_body(), X_O_cylinder, cylinder, "cylinder_visual", [0, 0, 1, 1])

# Finalize plant
plant.Finalize()

# Add Meshcat
meshcat = Meshcat()
visualizer = builder.AddSystem(MeshcatVisualizer(meshcat, scene_graph))
builder.Connect(scene_graph.get_query_output_port(), 
               visualizer.get_geometry_query_input_port())
diagram = builder.Build()
context = diagram.CreateDefaultContext()

In [ ]:

# === Define start and goal poses ===
q_start = np.array([0, 0, 0, 1, 0, 0, 0])  # [xyz, wxyz]
q_goal = np.array([1.0, 0.0, 1.0, 1, 0, 0, 0])

# === Plan geometric path using RRTConnect ===
planner = RRTConnect(plant, scene_graph)
result = planner.Solve(q_start, q_goal)
waypoints = result.get_state_trajectory().vector_values(result.get_state_trajectory().get_segment_times())

positions = np.array([wp[:3] for wp in waypoints])
s = np.linspace(0, 1, len(positions))

# === Create spline path for TOPPRA ===
cs_x = CubicSpline(s, positions[:, 0])
cs_y = CubicSpline(s, positions[:, 1])
cs_z = CubicSpline(s, positions[:, 2])

def path_eval(ss): return np.vstack((cs_x(ss), cs_y(ss), cs_z(ss))).T

path_obj = ta.SplineInterpolator(s, positions)

# === Add velocity and acceleration constraints ===
vlim = np.array([1.0, 1.0, 1.0])
alim = np.array([2.0, 2.0, 2.0])
v_constraint = constraint.JointVelocityConstraint(np.vstack([-vlim, vlim]))
a_constraint = constraint.JointAccelerationConstraint(np.vstack([-alim, alim]))

instance = algo.TOPPRA([v_constraint, a_constraint], path_obj, solver_wrapper='seidel')
jnt_traj = instance.compute_trajectory()

In [ ]:

# === Plot the result ===
ts_sample = np.linspace(0, jnt_traj.duration, 200)
qs_sample = jnt_traj(ts_sample)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(qs_sample[:, 0], qs_sample[:, 1], qs_sample[:, 2], label="TOPPRA trajectory")
ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2], label="Waypoints", c='r')
ax.legend()
plt.show()

In [ ]:
# === Meshcat Animation ===
cube_frame = plant.GetBodyByName("cube").body_frame()

vis_context = diagram.CreateDefaultContext()
vis_plant_context = plant.GetMyContextFromRoot(vis_context)
visualizer.load(vis_context)

pose_vec = []
for q in qs_sample:
    xyz = q
    R = RotationMatrix(RollPitchYaw(0, 0, 0))  # No rotation
    pose = RigidTransform(R, xyz)
    pose_vec.append(pose)

# Build time-sampled PiecewisePose trajectory
trajectory = PiecewisePose.MakeLinear(pose_vec, ts_sample)

# Animate
visualizer.get_mutable_output_port().Eval(vis_context)
visualizer.get_mutable_input_port().FixValue(vis_context, trajectory.value(0))

for t in np.linspace(0, trajectory.end_time(), 200):
    pose = trajectory.value(t)
    plant.SetFreeBodyPose(vis_plant_context, cube_body, pose)
    visualizer.ForcedPublish(vis_context)